In [1]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import StandardScaler, MinMaxScaler

import lightgbm as lgb
import catboost as cb
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor, GradientBoostingRegressor

from tsfresh.examples import load_robot_execution_failures
from tsfresh import extract_features, select_features
import optuna

from common import EP

import types

Using TensorFlow backend.


In [2]:
%%HTML
<style>
   div#notebook-container    { width: 95%; }
   div#menubar-container     { width: 65%; }
   div#maintoolbar-container { width: 99%; }
</style>

In [3]:
df_train = pd.read_pickle('../feats/df_train.pkl')
df_test = pd.read_pickle('../feats/df_test.pkl')

In [4]:
df_train['label'] = df_train['y'].apply(lambda x:  int(x) if x<15 else 15)
group = df_train['season'].values
group[np.where(group==17)[0]] = 1
df_train['group'] = group
df_train = df_train.drop(columns=['season'])

In [5]:
test_X = df_train.drop(columns=['y','index','group','label']).copy()
test_X.index = df_train['index']
test_y = df_train['y'].copy()
test_y.index = df_train['index']
tsfresh_columns = select_features(test_X, test_y).columns.tolist()

In [6]:
original_columns = df_train.columns.drop(['index','y','label','group']).tolist()

In [7]:
mytrial = []
# mytrial = list(pd.read_pickle('../trial/randomforest.pkl').T.to_dict().values())
# df_trial = pd.DataFrame(mytrial)
# len(mytrial)

In [12]:
param={
    'algorithm': {
        'cls': 'RandomForestRegressor',
        'fit': {
#             'early_stopping_rounds': 200,
#             'eval_metric': 'mae',
#             'verbose': False
        },
        'init': {
            'n_jobs':16,
        }
    },
    'columns': tsfresh_columns,
    'feature_importance': {
        'is_output': True,
        'permutation_feature_importance': True,
        'permutation_random_state': 1
    },
    'kfold': {
        'n_splits': 8,
        'random_state': 1985,
        'shuffle': True,
        'type': 'group'
    },
    'scaler': {
        'cls': 'StandardScaler'
    }
}

In [13]:
# run one try
df_his,  df_feature_importances, df_valid_pred, df_test_pred =  EP.process(df_train, param, df_test = None, trial=mytrial)

/home/ubuntu/anaconda3/envs/tensorflow_gpu_p36/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning:

The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.

/home/ubuntu/anaconda3/envs/tensorflow_gpu_p36/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning:

The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.

/home/ubuntu/anaconda3/envs/tensorflow_gpu_p36/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning:

The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.

/home/ubuntu/anaconda3/envs/tensorflow_gpu_p36/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning:

The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.

/home/ubuntu/anaconda3/envs/tensorflow_gpu_p36/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning:

The default value of n_estimators wil

In [16]:
#check feature_importances
df_feature_importances = df_trial.loc[0]['df_feature_importances']
if type(df_feature_importances)==pd.DataFrame:
    sorted_columns = EP.evaluate(df_feature_importances, key='average_model_weight')
else:
    sorted_columns = df_trial.loc[0]['param']['columns']
# df_feature_importances.sort_values(by=['average_permutation_weight'], ascending=False)

In [17]:
df_feature_importances.sort_values(by=['average_permutation_weight'], ascending=False).head()

,feature,model_weight,weight,std,model_weight1,weight1,std1,model_weight2,weight2,std2,...,weight5,std5,model_weight6,weight6,std6,model_weight7,weight7,std7,average_permutation_weight,average_model_weight
868,q25_roll_std_100,0.373545,0.444309,0.017157,0.206279,0.126971,0.004876,0.314034,0.333969,0.008993,...,1.010242,0.024856,0.283753,0.548885,0.012917,0.367678,0.606277,0.012505,0.456005,0.306904
954,spkt_welch_density__coeff_3,0.022846,0.048281,0.004707,0.016023,0.045081,0.004828,0.023321,0.067188,0.007275,...,0.073338,0.006221,0.023489,0.081961,0.007480,0.015251,0.032348,0.006348,0.056765,0.019988
449,abs_q01_4,0.001735,0.000738,0.000668,0.123328,0.057678,0.003314,0.122873,0.080708,0.003486,...,0.000844,0.001116,0.111704,0.061400,0.004007,0.040064,-0.052802,0.003765,0.048712,0.083460
464,abs_q25_5,0.047879,0.007450,0.001454,0.139827,0.084539,0.005107,0.007705,0.006157,0.000989,...,0.052525,0.003214,0.007196,0.016603,0.001178,0.018386,0.027545,0.001689,0.035441,0.033672
863,q05_roll_std_1000,0.029155,0.087159,0.003948,0.026011,0.008177,0.001874,0.035040,0.140079,0.003164,...,0.003123,0.001452,0.029140,0.001515,0.002721,0.032312,-0.056337,0.006403,0.023757,0.026516


In [18]:
param['columns']=sorted_columns[:200]
#  select features by permutation_weight
EP.select_features_(df_train, param, mytrial, nfeats_best=25, nfeats_removed_per_try=10, key='average_model_weight')

/home/ubuntu/anaconda3/envs/tensorflow_gpu_p36/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning:

The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.

/home/ubuntu/anaconda3/envs/tensorflow_gpu_p36/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning:

The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.

/home/ubuntu/anaconda3/envs/tensorflow_gpu_p36/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning:

The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.

/home/ubuntu/anaconda3/envs/tensorflow_gpu_p36/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning:

The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.

/home/ubuntu/anaconda3/envs/tensorflow_gpu_p36/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning:

The default value of n_estimators wil

/home/ubuntu/anaconda3/envs/tensorflow_gpu_p36/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning:

The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.

/home/ubuntu/anaconda3/envs/tensorflow_gpu_p36/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning:

The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.

/home/ubuntu/anaconda3/envs/tensorflow_gpu_p36/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning:

The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.

/home/ubuntu/anaconda3/envs/tensorflow_gpu_p36/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning:

The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.

/home/ubuntu/anaconda3/envs/tensorflow_gpu_p36/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning:

The default value of n_estimators wil

/home/ubuntu/anaconda3/envs/tensorflow_gpu_p36/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning:

The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.

/home/ubuntu/anaconda3/envs/tensorflow_gpu_p36/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning:

The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.

/home/ubuntu/anaconda3/envs/tensorflow_gpu_p36/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning:

The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.

/home/ubuntu/anaconda3/envs/tensorflow_gpu_p36/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning:

The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.

/home/ubuntu/anaconda3/envs/tensorflow_gpu_p36/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning:

The default value of n_estimators wil

/home/ubuntu/anaconda3/envs/tensorflow_gpu_p36/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning:

The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.

/home/ubuntu/anaconda3/envs/tensorflow_gpu_p36/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning:

The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.

/home/ubuntu/anaconda3/envs/tensorflow_gpu_p36/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning:

The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.

/home/ubuntu/anaconda3/envs/tensorflow_gpu_p36/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning:

The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.

/home/ubuntu/anaconda3/envs/tensorflow_gpu_p36/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning:

The default value of n_estimators wil

In [21]:
#[(df_trial['mae_diff']<.05)].sort_values(by=['val_mae'], ascending=True)
df_trial[['datetime','nfeatures', 'train_mae','train_mae_var','val_mae','val_mae_var','mae_diff']]

,datetime,nfeatures,train_mae,train_mae_var,val_mae,val_mae_var,mae_diff
0,2019-05-10 08:34:36.114938,1071,0.606334,0.000951,2.238428,0.468275,1.632094
1,2019-05-10 09:02:00.201596,200,0.561380,0.000912,2.248027,0.458334,1.686647
2,2019-05-10 09:16:17.318608,190,0.560413,0.000922,2.247255,0.454460,1.686841
3,2019-05-10 09:29:49.875937,180,0.557737,0.000936,2.255864,0.442800,1.698128
4,2019-05-10 09:42:38.611755,170,0.554183,0.000822,2.237563,0.453346,1.683380
5,2019-05-10 09:54:40.719433,160,0.552521,0.000915,2.256166,0.442115,1.703645
6,2019-05-10 10:05:57.953640,150,0.548946,0.000878,2.243773,0.446433,1.694827
7,2019-05-10 10:16:28.727436,140,0.546097,0.000947,2.247504,0.449189,1.701407
8,2019-05-10 10:26:13.069780,130,0.541811,0.000995,2.250985,0.458095,1.709173
9,2019-05-10 10:35:11.892170,120,0.539488,0.001015,2.248478,0.447397,1.708990


In [19]:
#  tune hypterparameters
def objective(trial):
        
    num_trees = trial.suggest_int('num_trees', 200, 1000)
    depth = trial.suggest_int('depth', 2, 10)
    learning_rate = trial.suggest_uniform('learning_rate', 0.01, 0.4)
    l2_leaf_reg = trial.suggest_uniform('l2_leaf_reg', 0.001, 100)
    bagging_temperature = trial.suggest_uniform('bagging_temperature', .6, 1)
    random_strength = trial.suggest_uniform('random_strength', .001, 1)
    random_state = trial.suggest_int('random_state', 1, 9999)
        
    args={
        'columns':df_trial.loc[235]['param']['columns'],
        'kfold':{
            'n_splits': 8,
            'random_state': 1985,
            'shuffle': True,
            'type': 'stratified'
        },
        'scaler':{
            'cls':'StandardScaler',
        },
        'algorithm':{
            'cls':'cb.CatBoostRegressor',
            'init':{
                "num_trees":num_trees,
                "depth":depth,
                "learning_rate":learning_rate,
                "l2_leaf_reg":l2_leaf_reg,
                "bagging_temperature":bagging_temperature,
                "random_strength":random_strength,
                "random_state":random_state,
            },
            'fit':{
#                 'eval_metric':'mae',
                'verbose':False,
                'early_stopping_rounds':200,
            },
        },
        'feature_importance':{
            'is_output':False,
            'permutation_feature_importance':False,
            'permutation_random_state':1,
        },
    }
    
    df_his,  df_feature_importances, df_valid_pred, df_test_pred =  EP.process(df_train, args, df_test = df_test, trial=mytrial, remark='tune feats selected by group ')
    val_mae_mean = np.mean(df_his.valid)
    val_mae_var = np.var(df_his.valid)
    train_mae_mean = np.mean(df_his.train)
    train_mae_var = np.var(df_his.train)
    
    trial.set_user_attr('val_mae', val_mae_mean)
    trial.set_user_attr('train_mae', train_mae_mean)
    trial.set_user_attr('mae_diff', val_mae_mean-train_mae_mean)
    trial.set_user_attr('val_mae_var', val_mae_var)

    return np.abs(val_mae_mean - train_mae_mean)*val_mae_mean

study = optuna.create_study()
study.optimize(objective, n_trials=200)

[I 2019-05-09 23:52:15,496] Finished a trial resulted in value: 0.23782823874437153. Current best value is 0.23782823874437153 with parameters: {'num_trees': 427, 'depth': 5, 'learning_rate': 0.38144574114176105, 'l2_leaf_reg': 59.8749840518477, 'bagging_temperature': 0.946313442752404, 'random_strength': 0.13930353388997965, 'random_state': 3775}.
[I 2019-05-10 00:01:05,266] Finished a trial resulted in value: 0.2653814536119777. Current best value is 0.23782823874437153 with parameters: {'num_trees': 427, 'depth': 5, 'learning_rate': 0.38144574114176105, 'l2_leaf_reg': 59.8749840518477, 'bagging_temperature': 0.946313442752404, 'random_strength': 0.13930353388997965, 'random_state': 3775}.
[I 2019-05-10 00:02:08,705] Finished a trial resulted in value: 0.3005346704633378. Current best value is 0.23782823874437153 with parameters: {'num_trees': 427, 'depth': 5, 'learning_rate': 0.38144574114176105, 'l2_leaf_reg': 59.8749840518477, 'bagging_temperature': 0.946313442752404, 'random_stre

[I 2019-05-10 01:00:24,242] Finished a trial resulted in value: 0.22789557033303964. Current best value is 0.015452438321236802 with parameters: {'num_trees': 671, 'depth': 3, 'learning_rate': 0.011444162636734707, 'l2_leaf_reg': 76.95163767459611, 'bagging_temperature': 0.8227980652968351, 'random_strength': 0.9784055466056077, 'random_state': 2054}.
[I 2019-05-10 01:03:54,712] Finished a trial resulted in value: 0.15591423359918127. Current best value is 0.015452438321236802 with parameters: {'num_trees': 671, 'depth': 3, 'learning_rate': 0.011444162636734707, 'l2_leaf_reg': 76.95163767459611, 'bagging_temperature': 0.8227980652968351, 'random_strength': 0.9784055466056077, 'random_state': 2054}.
[I 2019-05-10 01:10:33,902] Finished a trial resulted in value: 0.45636090127055284. Current best value is 0.015452438321236802 with parameters: {'num_trees': 671, 'depth': 3, 'learning_rate': 0.011444162636734707, 'l2_leaf_reg': 76.95163767459611, 'bagging_temperature': 0.8227980652968351, 

[I 2019-05-10 01:54:36,953] Finished a trial resulted in value: 0.20283246271929173. Current best value is 0.015452438321236802 with parameters: {'num_trees': 671, 'depth': 3, 'learning_rate': 0.011444162636734707, 'l2_leaf_reg': 76.95163767459611, 'bagging_temperature': 0.8227980652968351, 'random_strength': 0.9784055466056077, 'random_state': 2054}.
[I 2019-05-10 01:55:02,546] Finished a trial resulted in value: 0.082879097616782. Current best value is 0.015452438321236802 with parameters: {'num_trees': 671, 'depth': 3, 'learning_rate': 0.011444162636734707, 'l2_leaf_reg': 76.95163767459611, 'bagging_temperature': 0.8227980652968351, 'random_strength': 0.9784055466056077, 'random_state': 2054}.
[I 2019-05-10 01:56:16,063] Finished a trial resulted in value: 0.251055848826. Current best value is 0.015452438321236802 with parameters: {'num_trees': 671, 'depth': 3, 'learning_rate': 0.011444162636734707, 'l2_leaf_reg': 76.95163767459611, 'bagging_temperature': 0.8227980652968351, 'random

[I 2019-05-10 03:03:05,428] Finished a trial resulted in value: 0.02595757144358451. Current best value is 0.012773930232106656 with parameters: {'num_trees': 794, 'depth': 2, 'learning_rate': 0.011034654224196944, 'l2_leaf_reg': 95.69985890951777, 'bagging_temperature': 0.8326475639973028, 'random_strength': 0.8024641397038226, 'random_state': 937}.
[I 2019-05-10 03:04:09,466] Finished a trial resulted in value: 0.31165833678588917. Current best value is 0.012773930232106656 with parameters: {'num_trees': 794, 'depth': 2, 'learning_rate': 0.011034654224196944, 'l2_leaf_reg': 95.69985890951777, 'bagging_temperature': 0.8326475639973028, 'random_strength': 0.8024641397038226, 'random_state': 937}.
[I 2019-05-10 03:04:50,775] Finished a trial resulted in value: 0.13953294140595787. Current best value is 0.012773930232106656 with parameters: {'num_trees': 794, 'depth': 2, 'learning_rate': 0.011034654224196944, 'l2_leaf_reg': 95.69985890951777, 'bagging_temperature': 0.8326475639973028, 'r

[I 2019-05-10 03:40:55,386] Finished a trial resulted in value: 0.11516964066058367. Current best value is 0.007850106884887185 with parameters: {'num_trees': 303, 'depth': 4, 'learning_rate': 0.010304563320182062, 'l2_leaf_reg': 87.06643425778495, 'bagging_temperature': 0.9109520418792436, 'random_strength': 0.6357872506626183, 'random_state': 4485}.
[I 2019-05-10 03:41:11,817] Finished a trial resulted in value: 0.036896238730050566. Current best value is 0.007850106884887185 with parameters: {'num_trees': 303, 'depth': 4, 'learning_rate': 0.010304563320182062, 'l2_leaf_reg': 87.06643425778495, 'bagging_temperature': 0.9109520418792436, 'random_strength': 0.6357872506626183, 'random_state': 4485}.
[I 2019-05-10 03:42:44,438] Finished a trial resulted in value: 0.0657915107590635. Current best value is 0.007850106884887185 with parameters: {'num_trees': 303, 'depth': 4, 'learning_rate': 0.010304563320182062, 'l2_leaf_reg': 87.06643425778495, 'bagging_temperature': 0.9109520418792436, 

In [43]:
df_test_pred = df_trial.loc[342]['df_test_pred']

In [44]:
df_submit = pd.DataFrame()
df_submit['time_to_failure'] = np.mean(df_test_pred.drop(columns=['index']).values, axis=1)
df_submit['seg_id'] = df_test_pred['index']
df_submit.to_csv('submission.csv', index=False)

In [20]:
df_trial = pd.DataFrame(mytrial)

In [41]:
#[df_trial['mae_diff']<.05].sort_values(by=['val_mae']) | (df_trial['remark']=='tune feats selected by group ')
df_trial[(df_trial['remark']=='tune feats selected by group ')&(df_trial['mae_diff']<.05)].sort_values(by=['val_mae'], ascending=True)[['datetime','nfeatures', 'train_mae','train_mae_var','val_mae','val_mae_var','mae_diff']].head(10)

,datetime,nfeatures,train_mae,train_mae_var,val_mae,val_mae_var,mae_diff
342,2019-05-10 02:15:55.351409,30,1.822379,0.000016,1.870234,0.000263,0.047855
419,2019-05-10 03:39:47.282671,30,1.823879,0.000023,1.871669,0.000277,0.047790
325,2019-05-10 01:50:01.376788,30,1.825160,0.000022,1.875135,0.000253,0.049976
350,2019-05-10 02:23:27.277481,30,1.831164,0.000021,1.875346,0.000252,0.044182
260,2019-05-10 00:27:10.312556,30,1.830573,0.000013,1.876977,0.000200,0.046404
295,2019-05-10 01:18:30.771743,30,1.835525,0.000041,1.877723,0.000261,0.042197
299,2019-05-10 01:23:09.730396,30,1.831785,0.000015,1.878398,0.000167,0.046613
413,2019-05-10 03:36:48.905582,30,1.829911,0.000029,1.878617,0.000211,0.048706
307,2019-05-10 01:27:30.759789,30,1.831915,0.000010,1.878890,0.000221,0.046975
280,2019-05-10 00:56:29.669298,30,1.835339,0.000031,1.879742,0.000213,0.044403


In [42]:
df_trial.loc[342:342][['datetime','remark','nfeatures', 'train_mae','train_mae_var','val_mae','val_mae_var','mae_diff']]

,datetime,remark,nfeatures,train_mae,train_mae_var,val_mae,val_mae_var,mae_diff
342,2019-05-10 02:15:55.351409,tune feats selected by group,30,1.822379,0.000016,1.870234,0.000263,0.047855


In [46]:
df_trial.to_pickle('../trial/catboost.pkl')

In [47]:
df_trial.loc[342]['param']['columns']

['spkt_welch_density__coeff_3',
 'abs_q25_5',
 'q25_roll_std_100',
 'abs_q75_6',
 'abs_q75_7',
 'spkt_welch_densitycoeff_2',
 'abs_q01_4',
 'iqr_6',
 'q05_roll_std_100',
 'q05_roll_std_1000',
 'median__roll_std',
 'abs_q01_5',
 "number_peaks{'n': 10}",
 'FFT_Mag_75q0',
 "value_count{'value': 1}",
 'q01_roll_std_100',
 'abs_q95_2',
 'abs_q95_6',
 'MA_1000MA_std_mean_7',
 'q05_roll_std_10',
 'q01_roll_std_1000',
 'abs_max_roll_mean_1000',
 'abs_q75_2',
 'abs_q05_6',
 '5000std_quantile25',
 "number_crossing_m{'m': 1}",
 "autocorrelation{'lag': 5}",
 'q75_roll_std_10',
 'q05_2',
 '5000smoothness_quantile05']